In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(404290, 6)

In [4]:
df.sample(10)

,id,qid1,qid2,question1,question2,is_duplicate
108920,108920,91042,178861,Why is the rate of the expansion of the univer...,Why is the expansion of the universe accelerat...,1
362403,362403,4542,129243,How do the tourist attractions on the Scandina...,How do the tourist attractions on the Scandina...,0
3229,3229,6401,6402,What is the easiest way to live off the grid?,How is it living off the grid in India ?,0
176122,176122,604,193397,Do you need friends?,Do I need friends?,0
251003,251003,233898,28649,What's your favorite TV show and why?,Television Series: What is your favorite TV sh...,1
103933,103933,171675,171676,"If earth was a solid smooth ball, how deep wou...",Is the total amount of water on Earth always c...,0
124004,124004,200340,200341,How can one cope with the loss of a loved one?,How did you cope with the loss of a loved one?,1
237980,237980,349246,349247,What do pakistanis think about south indians?,What Pakistani people think about south Indian...,1
322749,322749,448617,366620,Does the Quora Top Writer get any kind of reco...,Do Top Writers on Quora get any rewards from t...,1
297874,297874,420291,420292,How do I make best use of hcverma in limited p...,I am using Evernote for UPSC notes. Is it limi...,0


In [5]:
new_df = df.sample(50000)

In [6]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [7]:
new_df.duplicated().sum()

0

In [8]:
ques_df = new_df[['question1', 'question2']]
ques_df.head()

,question1,question2
210161,Do you think scrapping of 500 and 1000 rupee d...,How will demonetization of 500 and 1000 rupee ...
102613,Is the real Wolf of Wall Street Jordan Belfort...,How accurate is The Wolf of Wall Street?
400541,To what extent was the Treaty of Versailles fa...,Was the Treaty of Versailles justifiable?
97092,What is the best universities for quantitative...,What is the best quantitative finance program ...
350427,What is a psychologist?,What does a psychologist do?


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
#merge text
questions = list(ques_df['question1']) + list(ques_df['question2'])

In [10]:
cv = CountVectorizer(max_features=5000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [11]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)

In [12]:
temp_df.shape

(50000, 10000)

In [13]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [14]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,4991,4992,4993,4994,4995,4996,4997,4998,4999,is_duplicate
210161,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
102613,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97092,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
350427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32883,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
214678,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281863,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:, 0:-1].values, temp_df.iloc[:, -1].values, 
                                                    test_size=0.2, random_state=42)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [18]:
rf = RandomForestClassifier()

In [19]:
rf.fit(X_train, y_train)

RandomForestClassifier()

In [20]:
y_pred = rf.predict(X_test)

In [21]:
accuracy_score(y_test, y_pred)

0.7556

In [25]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

In [26]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [27]:
y_pred = xgb.predict(X_test)
accuracy_score(y_test, y_pred)

0.7331